# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Not for Grading

## Rocchio feedback search updated (Pseudo feedback and Query Expansion)

In [ ]:
#@title Case Study Walkthrough
#@markdown   Rocchio feedback search updated (Pseudo feedback and Query Expansion)
from IPython.display import HTML

HTML("""<video width="320" height="240" controls>
  <source src="https://cdn.talentsprint.com/talentsprint/archives/sc/aiml/aiml_2018_b7_hyd/preview_videos/rocchio_feedback_search.mp4">
</video>
""")

In [ ]:
! wget https://cdn.talentsprint.com/aiml/CaseStudies/feedback_search.zip  
! unzip feedback_search.zip

In [ ]:
%cd feedback_search/

In [ ]:
%ls

##Implementing Rocchio feedback  search

Implementation of an information retrieval system that exploits user-provided relevance feedback to improve the search results returned by Google.


### Import required packages

In [ ]:
import sys
import logging
import threading
import os
import mock_feedback, mock_query_and_scraping
import query as query_file
import feedback
import enhance_query
import index
import scrape
import preprocess
import constants

### Setting up the logging for the feedback application with the required parameters

In [ ]:
# Create a logger
logger = logging.getLogger('feedback_search')
# Do not log in console
logger.propagate = False
# Create directory for logs if it's not there already
os.makedirs('logs', exist_ok=True) 
# Create handler
handler = logging.FileHandler('logs/feedback_search.log')
# Create formatter and add it to handler
formatter = logging.Formatter(
    fmt='[%(asctime)s %(levelname)s]\t%(message)s',
    datefmt='%d-%m-%Y %H:%M:%S')
handler.setFormatter(formatter)
# Add handler to the logger
logger.addHandler(handler)
# Create a set level to get INFO
logger.setLevel(logging.INFO)

For more information on logging module. Click on this [link](https://realpython.com/python-logging/).

**Note:** The below code executes untill it meets the precision value.

In [ ]:
def main(query, precision=0.8, is_test=False):
    """
    Main routine, 
    Takes initial query and target_precision provided as input,
    Until target_precision is achieved:
        Runs enhanced query, asks user's feedback, computes new precision.
    """

    try:
        target_precision = float(precision)
    except ValueError:
        print('<precision> must be a float between 0 and 1 !')
        return

    if target_precision > 1 or target_precision <0:
        print('<precision> must be a float between 0 and 1 !')
        return

    logger.info('\n\n ========================================================================\n\n')
    logger.info('[MAIN]\t\t Started with args: QUERY = %s, PRECISION = %s', query, target_precision)

    #import pdb; pdb.set_trace()
    achieved_precision = 0

    # Build one index for each zone of the documents (see enhance_query):
    indexers = {zone: index.UnigramIndexer(zone) for zone in ['title', 'summary', 'content']}
    bigram_indexers = {zone: index.BigramIndexer(zone) for zone in ['title', 'summary', 'content']}

    query_optimizer = enhance_query.RocchioQueryOptimizer()

    while (achieved_precision < target_precision):
        logger.info('[MAIN]\t\t achieved precision = %s vs target precision = %s, optimizing...', achieved_precision, target_precision)
        print('Parameters:')
        print('Query = {}'.format(query))
        print('Precision = {}'.format(target_precision))
        print('')
        
        if not is_test:
            results = query_file.query_google(query)

            # Fetch the whole documents by scraping the urls in results, as a background task
            scraping_thread = threading.Thread(target=scrape.add_url_content, args=(query, results))
            scraping_thread.start()

            if len(results) < 10:
                print('Too few results, aborting...')
                break

            # Ask feedback to user, store feedback in results dict directly
            feedback.ask_feedback(results)
            scraping_thread.join() # make sure all the documents have been scrapped

        elif is_test:
            results = mock_query_and_scraping.load_query_results(query)
            mock_feedback.mock_feedback(results, query=query)

        relevant = [doc['id'] for doc in results if doc['relevant']]
        non_relevant = [doc['id'] for doc in results if not doc['relevant']]
        achieved_precision = len(relevant)/len(results) if results else 0

        if achieved_precision == 0:
            print('Precision@10 is 0, aborting...')
            break

        logger.info('[MAIN]\t\t orginal query: %s', query)
        query = preprocess.split_remove_punctuation(query)
        if constants.USE_STEMMING:
            query = preprocess.stem(query)
        logger.info('[MAIN]\t\t preprocessed query: %s', query)

        indexing_threads = []
        for zone in indexers:
            indexers[zone].reset()
            t = threading.Thread(target=indexers[zone].index, args=(results, query))
            t.start()
            indexing_threads.append(t)
        
        for zone in bigram_indexers:
            bigram_indexers[zone].reset()
            t = threading.Thread(target=bigram_indexers[zone].index, args=(results, query))
            t.start()
            indexing_threads.append(t)

        for t in indexing_threads:
            t.join()

        print('Achieved precision: ', achieved_precision)

        query = query_optimizer.enhance(query, indexers, relevant, non_relevant, bigram_indexers=bigram_indexers)

        if is_test:
            # in tests we do Only run one of the query optimizer with mock feedback and analyze it
            return query

if __name__ == '__main__':
    main('research library journal study', precision=0.9)